# DSP Review
In this notebook we will introduce key ideas in digital signal processing such as:
1. Sampling
1. Frequency response


## Requirements
Create a virtual environment and install the requirements.

`pip install -r requirements.txt`


## Sampling
Most signals of interest start off as continuous signals. To process them on computers, we must sample them. This means discretize the time axis.

Let us simulate a continuous signal corresponiding to $x(t) = \sin(2\pi t)$ and sample it at regular intervals. The interval is known as the sampling period $T_s$. We use the following values
1. $T_s = 1$
1. $T_s = \frac{1}{2}$
1. $T_s = \frac{1}{4}$
1. $T_s = \frac{1}{8}$

We form a discrete time signal $x[n]$ where

$x[n]=x(t)\big|_{t=nT_s}$

$x[n]=x(nT_s)$

In [ ]:
# modify Ts below to see the effect on x[n]
import numpy as np
import matplotlib.pyplot as plt

t = np.linspace(-2, 2, 1000)
n = np.arange(-10,11)


Ts = 1/8
sampling_times = n * Ts

%matplotlib inline
plt.figure()
plt.plot(t, np.sin(2 * np.pi * t))
plt.xlabel('t', fontsize=20)
plt.ylim([-1.5, 1.5])
plt.ylabel(r'$x(t)$', fontsize=20)
plt.grid(True)

plt.figure()
plt.stem(n, np.sin(2 * np.pi * sampling_times), use_line_collection=True)
plt.ylim([-1.5, 1.5])
plt.xlabel(r'$n$', fontsize=20)
plt.xticks(np.arange(-10, 11), n)
plt.ylabel(r'$x[n]$', fontsize=20)
plt.grid(True)

### Audio signals
Often audio signals are stored as mp3s or wavs. These are still discrete sequences obtained by sampling and audio signal capture by a microphone at a given frequency $f_s = \frac{1}{T_s}$.

To record audio, we will use `sounddevice`. To process audio data, we will use the `librosa` library.

In [ ]:
# record audio
import sounddevice as sd

duration = 5  # seconds
fs = 16000 # 16kHz
myrecording = sd.rec(int(duration * fs), 
                     samplerate=fs, 
                     channels=1, 
                     dtype='int16')


sd.wait()

In [ ]:
max_audio_sample = 2 ** 15

plt.plot(np.arange(len(myrecording)) / fs, myrecording / max_audio_sample)
plt.xlabel(r'$t$', fontsize=20);
plt.ylabel(r'$s(t)$', fontsize=20);
plt.grid(True)

In [ ]:
# play recording
sd.play(myrecording, fs)

In [ ]:
128 * (1 / fs) 

In [ ]:
# get a block of samples and display
block_size = 128

blk_start = np.random.randint(0, 
                              len(myrecording) - block_size)

blk = myrecording[blk_start:blk_start + block_size]
plt.figure()
plt.stem(np.arange(block_size), 
         blk / max_audio_sample, 
         use_line_collection=True)
# plt.ylim([-1.5, 1.5])
plt.xlabel(r'$n$', fontsize=20)
plt.ylabel(r'$b[n]$', fontsize=20)
plt.grid(True)

## The Spectrum

A discrete time signal can be represented in both the time and frequency domains. In discrete time, the Fourier transform (DTFT) is computed as follows

$X(e^{j\omega})=\sum_{n=-\infty}^\infty x[n]e^{-j\omega n}$


We compute it using the function `freqz` from the `scipy.signal` module

In [ ]:
from scipy import signal

In [ ]:
signal.freqz?

Let's return to the sampled sinusoid and compute the frequency spectrum of the resulting sampled signal

In [ ]:
Ts = 1 /16
sampling_times = n * Ts
xn = np.sin(2 * np.pi * sampling_times)


plt.figure()
plt.stem(n, xn, use_line_collection=True)
plt.ylim([-1.5, 1.5])
plt.xlabel(r'$n$', fontsize=20)
plt.ylabel(r'$x[n]$', fontsize=20)
plt.grid(True)

plt.figure()
w, X = signal.freqz(xn)
plt.plot(w, abs(X), 'b')
plt.ylabel('Magnitude Response')
plt.xlabel('Frequency [rad/sample]')

Now let us consider the signal 

$x[n] = \cos(\omega_0n + 2\pi n) = \cos(\omega_0n)$

In [ ]:
np.mod(8, 5)

In [ ]:
np.pi / 8

In [ ]:
w0 = np.pi * .125   
xn = np.cos(w0 * n)

plt.figure()
plt.stem(n, xn, use_line_collection=True)
plt.ylim([-1.5, 1.5])
plt.xlabel(r'$n$', fontsize=20)
plt.ylabel(r'$x[n]$', fontsize=20)
plt.grid(True)

plt.figure()
w, X = signal.freqz(xn)
plt.plot(w, abs(X), 'b')
plt.axvline(np.mod(w0, 2 * np.pi) , c='r')
plt.ylabel('Magnitude Response')
plt.xlabel('Frequency [rad/sample]')


## The spectrum of sampled signals

When we sample a signal, we can use the sampled signal to estimate the frequency content of the original continuous time signal.

Consider the sinusoid 

\begin{equation}
x(t) = \cos(2\pi f_0 t)
\end{equation}

Let's generate this signal with $f_0 = 1$kHz. We will sample the signal at 16KHz.

In [ ]:
f0 = 1000
fs = 16e3
Ts = 1 / fs
duration = 5 # seconds
n = np.arange(duration * fs)

xn = np.concatenate((np.cos(2 * np.pi * f0 * n * Ts),
                     np.cos(2 * np.pi * 2 *f0 * n * Ts)))

plt.figure()
plt.stem(np.arange(2 * duration * fs), xn, use_line_collection=True)
plt.ylim([-1.5, 1.5])
plt.xlabel(r'$n$', fontsize=20)
plt.ylabel(r'$x[n]$', fontsize=20)
plt.grid(True)

In [ ]:
# play the tone - LOWER YOUR VOLUME BEFORE RUNNING
# change the frequency and repeat
sd.play(xn, fs)

## FFT

To estimate the spectrum of this signal, we will use the FFT. This is an efficient tool to compute the Discrete Fourier Transform of a discrete sequence. The DFT is a sampled version of the DTFT.

We start by taking a small block of N samples, taking the `fft` and plotting.

This FFT samples the spectrum at discrete frequencies $f_k$ between $0$ and $\frac{f_s}{2}$. We have

$f_k = \frac{kf_s}{N}$ for $k = 0, \ldots ,\frac{N}{2}$



In [ ]:
# get a block

nfft = 128

blk_start = np.random.randint(0, 
                              len(xn) - nfft)

blk = xn[blk_start:blk_start + nfft]
plt.figure()
plt.stem(np.arange(nfft), 
         blk, 
         use_line_collection=True)
plt.xlabel(r'$n$', fontsize=20)
plt.ylabel(r'$b[n]$', fontsize=20)
plt.grid(True)

In [ ]:
Xf = np.fft.fft(xn, nfft)

plt.plot(np.arange(nfft//2 + 1) / nfft * fs, 
         np.abs(Xf)[:nfft//2 + 1])
#plt.xlim([0, 5])
plt.xlabel('f', fontsize=20)
plt.ylabel(r'$\hat{X}(f)$', fontsize=20)
plt.xlim([0, fs / 2])
plt.grid(True)

In [ ]:
# if we compute the FFT for all blocks 
# we generate a spectrogram
import librosa
import librosa.display


hop_len = nfft // 2
spectrum = np.abs(librosa.stft(xn, 
                               n_fft=nfft, 
                               hop_length=hop_len))

plt.figure()
librosa.display.specshow(librosa.amplitude_to_db(spectrum, ref=np.max),
                         sr=fs,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')
plt.title('Spectrogram of sampled signal')
plt.xticks([])
plt.xlabel('')

In [ ]:
nfft = 256
hop_len = nfft // 2
spectrum = np.abs(librosa.stft(myrecording.reshape(1, -1)[0,:] / max_audio_sample, 
                               n_fft=nfft, 
                               hop_length=hop_len))

plt.figure()
librosa.display.specshow(librosa.amplitude_to_db(spectrum, ref=np.max),
                         sr=fs,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')
plt.title('Spectrogram of sampled signal')
plt.xticks([])
plt.xlabel('')

In [ ]:
myrecording.shape